In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
from collections import Counter
import random
import seaborn as sns

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import blockgroups in Boston URban Area
BUA_bg = pd.read_csv('drive/MyDrive/Amenities/output/geopandas_BUA_bgp.csv', index_col = False)
BUA_bg.drop('Unnamed: 0', axis=1, inplace=True)
BUA_bg.head(2)

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,boundary,centroid,intersects
0,25,17,353200,2,250173532002,Block Group 2,G5030,S,150628,0,42.357801,-71.105423,"POLYGON ((-71.109467 42.358362, -71.108813 42....","LINESTRING (-71.109467 42.358362, -71.108813 4...",POINT (-71.10542283047126 42.35780119427952),True
1,25,25,91100,5,250250911005,Block Group 5,G5030,S,66634,0,42.317050,-71.057945,"POLYGON ((-71.059544 42.316202, -71.0594579999...","LINESTRING (-71.059544 42.316202, -71.05945799...",POINT (-71.05794458903385 42.31705043327756),True


In [ ]:
#Import safegrap patterns & places
part1 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part1.csv', index_col = False)
part1.head(3)

,household_id,person_id,BLOCKGROUP,age
0,3588905919628999227,6006634466095081158,250092651011,88
1,10000071088832509892,13752919112610844558,230110110004,88
2,6470107995120863160,15149953993704830774,440070184003,88


In [ ]:
part2 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part2.csv', index_col = False)
part2.head(3)

,household_id,person_id,BLOCKGROUP,age
0,3588905919628999227,6006634466095081158,250092651011,88
1,10000071088832509892,13752919112610844558,230110110004,88
2,6470107995120863160,15149953993704830774,440070184003,88


In [ ]:
part3 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part3.csv', index_col = False)
part3.head(3)

,household_id,person_id,BLOCKGROUP,age
0,3588905919628999227,6006634466095081158,250092651011,88
1,10000071088832509892,13752919112610844558,230110110004,88
2,6470107995120863160,15149953993704830774,440070184003,88


In [ ]:
part4 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part4.csv', index_col = False)
part4.head(3)

,person_id,individual_income,household_size,household_income,family_structure,vehicles
0,3977447892636865643,17389,1_person,17389,living_alone,zero
1,11599401013491108390,58395,1_person,58395,living_alone,2
2,11253240366012147864,17676,1_person,17676,living_alone,1


In [ ]:
part5 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part5.csv', index_col = False)
part5.head(3)

,person_id,individual_income,household_size,household_income,family_structure,vehicles
0,3977447892636865643,17389,1_person,17389,living_alone,zero
1,11599401013491108390,58395,1_person,58395,living_alone,2
2,11253240366012147864,17676,1_person,17676,living_alone,1


In [ ]:
part6 = pd.read_csv ('drive/MyDrive/Amenities/data/bq_results/Copy of part6.csv', index_col = False)
part6.head(3)

,person_id,individual_income,household_size,household_income,family_structure,vehicles
0,3977447892636865643,17389,1_person,17389,living_alone,zero
1,11599401013491108390,58395,1_person,58395,living_alone,2
2,11253240366012147864,17676,1_person,17676,living_alone,1


In [ ]:
conc1 = pd.concat([part1, part2, part3]).reset_index().drop('index', axis = 1)
conc1= conc1[conc1['BLOCKGROUP'].isin(BUA_bg['GEOID'])].reset_index().drop('index', axis = 1)
conc1.head(2)

,household_id,person_id,BLOCKGROUP,age
0,7670780544235756806,12084962145891878465,250173537002,88
1,12636066713864596213,17075063108892551063,250214010003,88


In [ ]:
conc2 = pd.concat([part4, part5, part6]).reset_index().drop('index', axis = 1)
table=conc1.merge(conc2,how='left',on='person_id')
table.head(2)

,household_id,person_id,BLOCKGROUP,age,individual_income,household_size,household_income,family_structure,vehicles
0,7670780544235756806,12084962145891878465,250173537002,88,178836,2_person,244316,nonfamily_single,1
1,7670780544235756806,12084962145891878465,250173537002,88,178836,2_person,244316,nonfamily_single,1


In [ ]:
table['person_id'].nunique()

551387

In [ ]:
table=table.drop_duplicates(subset=['person_id']).reset_index().drop('index', axis = 1)
table.head(2)

,household_id,person_id,BLOCKGROUP,age,individual_income,household_size,household_income,family_structure,vehicles
0,7670780544235756806,12084962145891878465,250173537002,88,178836,2_person,244316,nonfamily_single,1
1,12636066713864596213,17075063108892551063,250214010003,88,54048,3_person,80556,married_couple,1


In [ ]:
table.to_csv('drive/MyDrive/Amenities/output/bigquery_BUA_population.csv', encoding='utf-8', index=False)

In [ ]:
len(table)

551387